In [4]:
from PIL import Image
import os
import numpy as np
import pandas as pd
import houghBoxRotate as hbr
import matplotlib.pyplot as plt
import cv2

In [5]:
def apply_clahe(image):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    clahe_image = clahe.apply(image)

    return clahe_image

def draw_lines(image):
    edges = cv2.Canny(image, threshold1=50, threshold2=150)

    # Find lines using Hough Line Transform
    lines = cv2.HoughLines(edges, rho=1, theta=np.pi / 180, threshold=100)
    if lines is None:
        return image
    # Draw lines on the original image
    for line in lines:
        rho, theta = line[0]
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        x1 = int(x0 + 1000 * (-b))
        y1 = int(y0 + 1000 * (a))
        x2 = int(x0 - 1000 * (-b))
        y2 = int(y0 - 1000 * (a))
        cv2.line(image, (x1, y1), (x2, y2), (255), 2)  # Draw red lines

    return image

def rotate_image(image):
    # Detect edges using Canny edge detection
    edges = cv2.Canny(image, threshold1=50, threshold2=150)

    # Find lines in the image using Hough Transform
    lines = cv2.HoughLines(edges, rho=1, theta=np.pi/180, threshold=100)

    # Identify the orientation angle from the detected lines
    total_angle = 0
    mean_angle = 0
    if lines is not None and len(lines) > 0:
        for line in lines:
            rho, theta = line[0]
            angle = theta * (180 / np.pi) # Convert theta to degrees and adjust to vertical
            total_angle += angle
        mean_angle = total_angle / len(lines)
    
    # Get the image dimensions
    height, width = image.shape[:2]

    # Calculate the rotation matrix
    rotation_matrix = cv2.getRotationMatrix2D((width / 2, height / 2), mean_angle, 1.0)

    # Apply the rotation using warpAffine
    rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0))


    # rotated_image = draw_lines(rotated_image)

    return rotated_image

def invert_image(image):
    return cv2.bitwise_not(image)


In [7]:
images = []
invert_images = []
# Open to_invert.txt and loop through each line
with open('to_invert.txt') as f:
    for line in f:
        invert_images.append(line.strip())

for i, image_file in enumerate(os.listdir('datasets/finalized_dataset')):
    
    # Read image
    img = Image.open('datasets/finalized_dataset/' + image_file)
    img = np.array(img.convert('L'))
    img_copy = img.copy()
    # Check if image_file contains ".png.png"
    if image_file in invert_images:
        img = invert_image(img)

    img = apply_clahe(img)
    img = rotate_image(img)

    # Put two images side-by-side
    img_copy = cv2.cvtColor(img_copy, cv2.COLOR_BGR2RGB)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Make images have same size
    if img.shape[0] > img_copy.shape[0]:
        img_copy = cv2.resize(img_copy, (img.shape[1], img.shape[0]))
    else:
        img = cv2.resize(img, (img_copy.shape[1], img_copy.shape[0]))

    vis = np.concatenate((img_copy, img), axis=1)
    # save image
    cv2.imwrite('datasets/edited/' + image_file, vis)
    images.append(img)
# # Plot the images
# fig, ax = plt.subplots(4,4, figsize=(15,15))
# for i in range(len(images)):
#     ax[i//4, i%4].imshow(images[i], cmap='gray')